**Object Detection with YOLO and OpenCV**

In [1]:
pip install ultralytics opencv-python pyyaml pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.1 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 4.4 MB/s  0:00:00
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ------ --------------------------------- 6.8/39.0 MB 32.1 MB/s eta 0:00:02
   ------------- -------------------------- 13.4/39.0 MB 32.2 MB/s eta 0:00:01
   -------------------- ------------------- 20.2/39.0 MB 31.9 MB/s eta 0:00:01
   --------------------------- ------------ 27.3/39.0 MB 31.4 MB/s eta 0:00:01
   ---------------------------------- ----- 34.1/39.0 MB 31.3 MB/s eta 0:00:01
   ---------------------------------------  38.8/39.0 MB 30.8 MB/s eta 0:00:01
   ---------------------------------------  38.8/39.0 MB 30.8 MB/s eta 0:00:01
   ---------------------------------------  38.8/39.0 MB 30.8 MB/s eta 0:00:01
   

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install ultralytics opencv-python

import cv2
import random
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# Load Yolo Model
# yolov8n.pt: Nano (fastest, less accurate)
# yolov8x.pt: Extra Large (most accurate, slower)

yolo = YOLO("yolov8s.pt")

# Define Function to Assign Colors
def getColours(cls_num):
    """Generate unique colors for each class ID"""
    random.seed(cls_num)
    return tuple(random.randint(0, 255) for _ in range(3))

# Load the Video
video_path = "your_video.mp4"
videoCap = cv2.VideoCapture(video_path)

#Process Video and Detect Objects
frame_count = 0

while True:
    ret, frame = videoCap.read()
    if not ret:
        break
    results = yolo.track(frame, stream=True)

    for result in results:
        class_names = result.names
        for box in result.boxes:
            if box.conf[0] > 0.4:
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                cls = int(box.cls[0])
                class_name = class_names[cls]

                conf = float(box.conf[0])

                colour = getColours(cls)

                cv2.rectangle(frame, (x1, y1), (x2, y2), colour, 2)

                cv2.putText(frame, f"{class_name} {conf:.2f}",
                            (x1, max(y1 - 10, 20)), cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, colour, 2)

    if frame_count < 20:
        cv2_imshow(frame)
    else:
        break

    frame_count += 1

videoCap.release()

ChatGPT

In [ ]:
"""
YOLO (Ultralytics) reference block:
- Validate YOLO dataset structure
- Create data.yaml
- Sanity check label files
- Train / Predict
- Visualize predictions and training metrics
"""

from pathlib import Path
import yaml
import cv2
import pandas as pd
import matplotlib.pyplot as plt

from ultralytics import YOLO


# =========================
# DATA PREPARATION
# =========================

def validate_yolo_dataset(root: Path) -> None:
    """
    Validate YOLO dataset structure:
      root/images/train, root/images/val
      root/labels/train, root/labels/val
    """
    required = [
        root / "images/train",
        root / "images/val",
        root / "labels/train",
        root / "labels/val",
    ]

    missing = [p for p in required if not p.exists()]
    if missing:
        raise FileNotFoundError(
            "Missing required paths:\n" + "\n".join(str(p) for p in missing)
        )

    print("YOLO dataset structure is valid:", root)


def create_data_yaml(
    dataset_root: Path,
    class_names: list[str],
    output_path: Path
) -> None:
    """
    Create YOLO data.yaml file.
    """
    data = {
        "path": str(dataset_root),
        "train": "images/train",
        "val": "images/val",
        "names": {i: name for i, name in enumerate(class_names)},
    }

    output_path.parent.mkdir(parents=True, exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        yaml.safe_dump(data, f, sort_keys=False, allow_unicode=True)

    print("Created data.yaml at:", output_path)


def check_yolo_labels(
    labels_dir: Path,
    num_classes: int,
    sample_k: int = 3
) -> None:
    """
    Sanity check YOLO label files.
    Format per line:
      class_id x_center y_center width height
    """
    label_files = sorted(labels_dir.glob("*.txt"))
    if not label_files:
        raise FileNotFoundError(f"No label files found in {labels_dir}")

    issues = 0
    for lf in label_files:
        with open(lf, "r", encoding="utf-8") as f:
            for ln, line in enumerate(f, 1):
                parts = line.strip().split()
                if not parts:
                    continue

                if len(parts) != 5:
                    print(f"Invalid format in {lf.name}:{ln}")
                    issues += 1
                    continue

                cls = int(float(parts[0]))
                if cls < 0 or cls >= num_classes:
                    print(f"Invalid class id {cls} in {lf.name}:{ln}")
                    issues += 1

                coords = list(map(float, parts[1:]))
                if any(c < 0 or c > 1 for c in coords):
                    print(f"Invalid normalized coords in {lf.name}:{ln}")
                    issues += 1

    if issues > 0:
        raise ValueError(f"Found {issues} issues in label files")

    print(f"Label files are valid in {labels_dir}")

    for lf in label_files[:sample_k]:
        print(f"\nSample from {lf.name}:")
        print(lf.read_text(encoding="utf-8")[:300])


# =========================
# TRAINING AND INFERENCE
# =========================

def train_yolo(
    data_yaml: Path,
    model_name: str = "yolov8n.pt",
    epochs: int = 20,
    imgsz: int = 640,
    batch: int = 16,
    optimizer: str = "Adam",
    project: str = "runs",
    name: str = "exp",
    device: int | str = 0,
):
    """
    Train YOLO model using Ultralytics API.
    """
    model = YOLO(model_name)

    results = model.train(
        data=str(data_yaml),
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        optimizer=optimizer,
        project=project,
        name=name,
        device=device,
    )

    return results


def predict_yolo(
    weights: Path,
    source,
    conf: float = 0.25,
    imgsz: int = 640,
    save: bool = True,
    project: str | None = None,
    name: str | None = None,
):
    """
    Run YOLO inference.
    """
    model = YOLO(str(weights))

    results = model.predict(
        source=source,
        conf=conf,
        imgsz=imgsz,
        save=save,
        project=project,
        name=name,
    )

    return results


# =========================
# VISUALIZATION
# =========================

def show_yolo_prediction(
    image_path: Path,
    yolo_result,
    class_names: list[str] | None = None
) -> None:
    """
    Visualize YOLO predictions on a single image.
    """
    img = cv2.imread(str(image_path))
    if img is None:
        raise FileNotFoundError(f"Cannot read image
